# 前言
进入深度学习后，因为使用的包不同，对待模型的原理不再是像sklearn一样简单创建一个xx器（预测器、预处理器……），再使用内置的方法，而是先构建使用包方法构建模型结构，再使用相应方法迭代训练。  
所有的代码，为可重复性，都应该加入随机种子。参数具体是random_state = ...，为节省空间，下述不再写作。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import tqdm
import sys

# 强化学习
强化学习是深度学习里面非常特别的一类。很明显地，该数据科学方法引入了心理学行为主义/应用行为分析的思想，使用了“强化”、“奖励”等多个概念来对数据进行多轮次训练。如果说之前的数据科学（机器学习、批量学习）可以说还在建模的框架内，与科学研究中的“真理”推断只是稍微有些差异（稍微转向预测）；那么从强化学习开始，数据科学则走向了更严格的，纯粹为了目的服务的方向。  
强化学习的核心思想是探索与利用（Exploration and Exploitation）。所谓的探索即是科研，即得到“真理”，建构对世界的认识，但不利于最后的结果产出（最大化收益）。商业上的ABtest其实就是随机对照试验RCT，属于科研精神的一环，而强化学习所强调的探索与利用的平衡，则是“算法精神”起作用，纯粹为了目的服务。  

In [ ]:
# ϵ−贪心算法 Epsilon-Greedy Algorithm
  
# Define Action class
class Actions:
    def __init__(self, m):
        self.m = m
        self.mean = 0
        self.N = 0

    # Choose a random action
    def choose(self): 
        return np.random.randn() + self.m

    # Update the action-value estimate
    def update(self, x):
        self.N += 1
        self.mean = (1 - 1.0 / self.N) * self.mean + 1.0 / self.N * x

def run_experiment(m1, m2, m3, eps, N):
    actions = [Actions(m1), Actions(m2), Actions(m3)]
    data = np.empty(N)
    for i in range(N):   # epsilon greedy
        p = np.random.random()
        if p < eps:
            j = np.random.choice(3)
        else:
            j = np.argmax([a.mean for a in actions])
        x = actions[j].choose()
        actions[j].update(x)
    
    # for the plot
    data[i] = x
    cumulative_average = np.cumsum(data) / (np.arange(N) + 1)
    
    # plot moving average ctr
    plt.plot(cumulative_average)
    plt.plot(np.ones(N)*m1)
    plt.plot(np.ones(N)*m2)
    plt.plot(np.ones(N)*m3)
    plt.xscale('log')
    plt.show()
    
    for a in actions:
        print(a.mean)
    return cumulative_average

if __name__ == '__main__':
    c_1 = run_experiment(1.0, 2.0, 3.0, 0.1, 100000)
    c_05 = run_experiment(1.0, 2.0, 3.0, 0.05, 100000)
    c_01 = run_experiment(1.0, 2.0, 3.0, 0.01, 100000)

In [ ]:
# 置信区间上界算法

import matplotlib.pyplot as plt
import pandas as pd
import math

# import the dataset
dataset = pd.read_csv('Ads_CTR_Optimisation.csv')

# Implementing UCB
N = 10000
d = 10
ads_selected = []
numbers_of_selections = [0] * d
sums_of_rewards = [0] * d
total_reward = 0
for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if numbers_of_selections[i] > 0:
            average_reward = sums_of_rewards[i] / numbers_of_selections[i]
            delta_i = math.sqrt(3/2 * math.log(n + 1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    numbers_of_selections[ad] = numbers_of_selections[ad] + 1
    sums_of_rewards[ad] = sums_of_rewards[ad] + reward
    total_reward = total_reward + reward

# Visualising the results
plt.hist(ads_selected)
plt.title('Histogram of ads selections')
plt.xlabel('Ads')
plt.ylabel('Number of times each ad was selected')
plt.show()

In [ ]:
# 汤普森采样/伯努利-汤普森采样/高斯-汤普森采样/softmax-汤普森采样

def thompson_sampling(env, 
                      alpha=1,
                      beta=0,
                      n_episodes=1000):
    Q = np.zeros((env.action_space.n), dtype=np.float64)
    N = np.zeros((env.action_space.n), dtype=np.int)
    
    Qe = np.empty((n_episodes, env.action_space.n), dtype=np.float64)
    returns = np.empty(n_episodes, dtype=np.float64)
    actions = np.empty(n_episodes, dtype=np.int)
    name = 'Thompson Sampling {}, {}'.format(alpha, beta)
    for e in tqdm(range(n_episodes), 
                  desc='Episodes for: ' + name, 
                  leave=False):
        samples = np.random.normal(
            loc=Q, scale=alpha/(np.sqrt(N) + beta))
        action = np.argmax(samples)

        _, reward, _, _ = env.step(action)
        N[action] += 1
        Q[action] = Q[action] + (reward - Q[action])/N[action]

        Qe[e] = Q
        returns[e] = reward
        actions[e] = action
    return name, returns, Qe, actions

class GaussianThompsonSocket():   # 本部分类基于他人的代码，使用高斯汤普森类方法处理它自己的机器人代码对象。后续应把该串代码整理成函数形式。
    def __init__(self, q):                
                
        self.τ_0 = 0.0001  # the posterior precision
        self.μ_0 = 1       # the posterior mean
        
        # pass the true reward value to the base PowerSocket             
        super().__init__(q)         
        
    def sample(self):
        """ return a value from the the posterior normal distribution """
        return (np.random.randn() / np.sqrt(self.τ_0)) + self.μ_0    
                    
    def update(self,R):
        """ update this socket after it has returned reward value 'R' """   

        # do a standard update of the estimated mean
        super().update(R)    
               
        # update the mean and precision of the posterior
        self.μ_0 = ((self.τ_0 * self.μ_0) + (self.n * self.Q))/(self.τ_0 + self.n)        
        self.τ_0 += 1      

    def charge(self):
        """ return a random amount of charge """
        # the reward is a guassian distribution with unit variance around the true value 'q'
        value = np.random.randn() + self.q 

def softmax(env, 
            init_temp=float('inf'), 
            min_temp=0.0,
            decay_ratio=0.04,
            n_episodes=1000):
    Q = np.zeros((env.action_space.n), dtype=np.float64)
    N = np.zeros((env.action_space.n), dtype=np.int)

    Qe = np.empty((n_episodes, env.action_space.n), dtype=np.float64)
    returns = np.empty(n_episodes, dtype=np.float64)
    actions = np.empty(n_episodes, dtype=np.int)
    name = 'Lin SoftMax {}, {}, {}'.format(init_temp, 
                                           min_temp,
                                           decay_ratio)
    # can't really use infinity
    init_temp = min(init_temp,
                    sys.float_info.max)
    # can't really use zero
    min_temp = max(min_temp,
                   np.nextafter(np.float32(0), 
                                np.float32(1)))
    for e in tqdm(range(n_episodes),
                  desc='Episodes for: ' + name, 
                  leave=False):
        decay_episodes = n_episodes * decay_ratio
        temp = 1 - e / decay_episodes
        temp *= init_temp - min_temp
        temp += min_temp
        temp = np.clip(temp, min_temp, init_temp)

        scaled_Q = Q / temp
        norm_Q = scaled_Q - np.max(scaled_Q)
        exp_Q = np.exp(norm_Q)
        probs = exp_Q / np.sum(exp_Q)
        assert np.isclose(probs.sum(), 1.0)

        action = np.random.choice(np.arange(len(probs)), 
                                  size=1, 
                                  p=probs)[0]

        _, reward, _, _ = env.step(action)
        N[action] += 1
        Q[action] = Q[action] + (reward - Q[action])/N[action]
        
        Qe[e] = Q
        returns[e] = reward
        actions[e] = action
    return name, returns, Qe, actions

# 深度学习：基本PyTorch

In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary # torchsummary不支持lstm
from torchinfo import summary as info_summary

#### 判断当前计算平台

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print('当前计算平台：', device)

当前计算平台： cuda


## 定义模型框架

In [3]:
class MyModel(nn.Module):  # define the CNN architecture
    def __init__(self, in_channel=1, small_Fs=8, big_Fs=6, Fs=100):
        super().__init__()

        self.smallCNN = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=Fs//2, stride =Fs//16, padding=Fs//2//2, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(True),

            nn.MaxPool1d(kernel_size=8, stride=8,  padding=4),
            nn.Dropout(0.5),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=small_Fs,
                      stride=1, padding=small_Fs//2, bias =False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=small_Fs,
                      stride=1, padding=small_Fs//2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=small_Fs,
                      stride=1, padding=small_Fs//2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.MaxPool1d(kernel_size=4, stride=4, padding=2)
        )

        self.bigCNN = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=Fs*4, stride=Fs//2, padding=Fs*4//2, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(True),

            nn.MaxPool1d(kernel_size=4, stride=4, padding=2),  # padding = filter_size//2
            nn.Dropout(0.5),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=big_Fs, stride=1, padding=big_Fs//2, bias= False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=big_Fs, stride=1, padding=big_Fs//2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.Conv1d(in_channels =128, out_channels=128, kernel_size=big_Fs, stride=1, padding=big_Fs//2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1280 + 2176, 6)  # big and small conv concat
            # 貌似和输入数据长度有关，1280+2176适用于长度3000，若要改为长度7500，需修改此处
        )

    def forward(self, x):
        small = self.smallCNN(x)
        big = self.bigCNN(x)

        feature_big = torch.flatten(big, 1)
        feature_small = torch.flatten(small, 1)  # (batch, channel)
        output = torch.cat((feature_big, feature_small), dim=1)
        output = self.fc(output)
        return(output)


# 配置模型并配置在当前计算平台
model = MyModel()
model.to(device)

criterion = nn.CrossEntropyLoss().to(device)  # 使用 CrossEntropy 作为 Cost 函数
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=[0.9, 0.999])  # 使用基础Adam作为Optimizier（优化器）

print('【模型参数】')
summary(model,(1,3000))

epochs = 100  # 重复次数为 100
cnt = 0  # 用于判断early stopping

# 训练和验证的loss（损失）值
train_loss = torch.zeros(epochs)
val_loss = torch.zeros(epochs)

# 训练和验证的acc（准确度）值
train_acc = torch.zeros(epochs)
val_acc = torch.zeros(epochs)

# 初始loss是无穷大
valid_loss_min = np.Inf

【模型参数】
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 128, 17]             --
|    └─Conv1d: 2-1                       [-1, 64, 501]             3,200
|    └─BatchNorm1d: 2-2                  [-1, 64, 501]             128
|    └─ReLU: 2-3                         [-1, 64, 501]             --
|    └─MaxPool1d: 2-4                    [-1, 64, 63]              --
|    └─Dropout: 2-5                      [-1, 64, 63]              --
|    └─Conv1d: 2-6                       [-1, 128, 64]             65,536
|    └─BatchNorm1d: 2-7                  [-1, 128, 64]             256
|    └─ReLU: 2-8                         [-1, 128, 64]             --
|    └─Conv1d: 2-9                       [-1, 128, 65]             131,072
|    └─BatchNorm1d: 2-10                 [-1, 128, 65]             256
|    └─ReLU: 2-11                        [-1, 128, 65]             --
|    └─Conv1d: 2-12                      [-1, 128, 66]         

## 迁移学习

In [ ]:
### 保存一个原始模型，做测试用
trans_model = model
save_model_path = "./model_best.pt"
### 导入需要的模型
save_model = torch.load(save_model_path)

### 设定要删除的key
all_keys = ['smallCNN.0.weight', 'smallCNN.1.weight', 'smallCNN.1.bias', 'smallCNN.1.running_mean', 'smallCNN.1.running_var', 'smallCNN.1.num_batches_tracked', 'smallCNN.5.weight', 'smallCNN.6.weight', 'smallCNN.6.bias', 'smallCNN.6.running_mean', 'smallCNN.6.running_var', 'smallCNN.6.num_batches_tracked', 'smallCNN.8.weight', 'smallCNN.9.weight', 'smallCNN.9.bias', 'smallCNN.9.running_mean', 'smallCNN.9.running_var', 'smallCNN.9.num_batches_tracked', 'smallCNN.11.weight', 'smallCNN.12.weight', 'smallCNN.12.bias', 'smallCNN.12.running_mean', 'smallCNN.12.running_var', 'smallCNN.12.num_batches_tracked', 'bigCNN.0.weight', 'bigCNN.1.weight', 'bigCNN.1.bias', 'bigCNN.1.running_mean', 'bigCNN.1.running_var', 'bigCNN.1.num_batches_tracked', 'bigCNN.5.weight', 'bigCNN.6.weight', 'bigCNN.6.bias', 'bigCNN.6.running_mean', 'bigCNN.6.running_var', 'bigCNN.6.num_batches_tracked', 'bigCNN.8.weight', 'bigCNN.9.weight', 'bigCNN.9.bias', 'bigCNN.9.running_mean', 'bigCNN.9.running_var', 'bigCNN.9.num_batches_tracked', 'bigCNN.11.weight', 'bigCNN.12.weight', 'bigCNN.12.bias', 'bigCNN.12.running_mean', 'bigCNN.12.running_var', 'bigCNN.12.num_batches_tracked','fc.1.weight', 'fc.1.bias']
freeze_keys = ['smallCNN.0.weight', 'smallCNN.1.weight', 'smallCNN.1.bias', 'smallCNN.1.running_mean', 'smallCNN.1.running_var', 'smallCNN.1.num_batches_tracked', 'smallCNN.5.weight', 'smallCNN.6.weight', 'smallCNN.6.bias', 'smallCNN.6.running_mean', 'smallCNN.6.running_var', 'smallCNN.6.num_batches_tracked', 'smallCNN.8.weight', 'smallCNN.9.weight', 'smallCNN.9.bias', 'smallCNN.9.running_mean', 'smallCNN.9.running_var', 'smallCNN.9.num_batches_tracked', 'smallCNN.11.weight', 'smallCNN.12.weight', 'smallCNN.12.bias', 'smallCNN.12.running_mean', 'smallCNN.12.running_var', 'smallCNN.12.num_batches_tracked', 'bigCNN.0.weight', 'bigCNN.1.weight', 'bigCNN.1.bias', 'bigCNN.1.running_mean', 'bigCNN.1.running_var', 'bigCNN.1.num_batches_tracked', 'bigCNN.5.weight', 'bigCNN.6.weight', 'bigCNN.6.bias', 'bigCNN.6.running_mean', 'bigCNN.6.running_var', 'bigCNN.6.num_batches_tracked', 'bigCNN.8.weight', 'bigCNN.9.weight', 'bigCNN.9.bias', 'bigCNN.9.running_mean', 'bigCNN.9.running_var', 'bigCNN.9.num_batches_tracked', 'bigCNN.11.weight', 'bigCNN.12.weight', 'bigCNN.12.bias', 'bigCNN.12.running_mean', 'bigCNN.12.running_var', 'bigCNN.12.num_batches_tracked']
delete_keys = list(set(all_keys).difference(set(freeze_keys)))

### 传入参数
for para in delete_keys:
    del save_model[para]
trans_model_dict = trans_model.state_dict()
state_dict = {k:v for k,v in save_model.items() if k in trans_model_dict.keys()}
trans_model_dict.update(state_dict)
trans_model.load_state_dict(trans_model_dict)

### 冻结参数
for name, parameter in trans_model.named_parameters():
    for key in freeze_keys:
        if key in name:
            parameter.requires_grad = False


## 暂时无用的代码

In [ ]:
# 单个切片的目标长度
len_slice = 3000

# 单个切片的有效数据长度
len_slice_useful = 2800

# 单侧补零长度
len_slice_addzeros_oneside = int((len_slice - len_slice_useful)/2)

# 单侧补零模板
zeros_oneside = [0.5]*len_slice_addzeros_oneside

# 各分期的数据文件保存位置，以及迁移学习的best_model_path
path_W = 'E:/SleepEEGNet/DeepSleepNet_SM/data_SM/W'
path_L = 'E:/SleepEEGNet/DeepSleepNet_SM/data_SM/L'
path_D = 'E:/SleepEEGNet/DeepSleepNet_SM/data_SM/D'
path_R = 'E:/SleepEEGNet/DeepSleepNet_SM/data_SM/R'
save_model_path = 'E:/SleepEEGNet/DeepSleepNet_SM/sample/model_best.pt'

# 各分期的EEG文件列表
datalist_W = glob.glob(path_W + '/**')
datalist_L = glob.glob(path_L + '/**')
datalist_D = glob.glob(path_D + '/**')
datalist_R = glob.glob(path_R + '/**')

# 各分期的切片数量
num_W = len(datalist_W)
num_L = len(datalist_L)
num_D = len(datalist_D)
num_R = len(datalist_R)
num_all = num_W + num_L + num_D + num_R

# 将各分期EEG数据归一化、前后补零，然后分类汇总（拼接）
data_W = []
for idx in range(num_W):
    current_data = np.loadtxt(datalist_W[idx], dtype=np.float32, delimiter=',')
    current_data = current_data[:len_slice_useful]

    # 数据归一化
    current_data = (current_data - np.min(current_data)) / (np.max(current_data) - np.min(current_data))

    # 数据切片前后补零
    current_data = np.array(zeros_oneside + list(current_data) + zeros_oneside)
    data_W.append(current_data)

data_L = []
for idx in range(num_L):
    current_data = np.loadtxt(datalist_L[idx], dtype=np.float32, delimiter=',')
    current_data = current_data[:len_slice_useful]

    # 数据归一化
    current_data = (current_data - np.min(current_data)) / (np.max(current_data) - np.min(current_data))

    # 数据切片前后补零
    current_data = np.array(zeros_oneside + list(current_data) + zeros_oneside)
    data_L.append(current_data)

data_D = []
for idx in range(num_D):
    current_data = np.loadtxt(datalist_D[idx], dtype=np.float32, delimiter=',')
    current_data = current_data[:len_slice_useful]

    # 数据归一化
    current_data = (current_data - np.min(current_data)) / (np.max(current_data) - np.min(current_data))

    # 数据切片前后补零
    current_data = np.array(zeros_oneside + list(current_data) + zeros_oneside)

    data_D.append(current_data)

data_R = []
for idx in range(num_R):
    current_data = np.loadtxt(datalist_R[idx], dtype=np.float32, delimiter=',')
    current_data = current_data[:len_slice_useful]

    # 数据归一化
    current_data = (current_data - np.min(current_data)) / (np.max(current_data) - np.min(current_data))

    # 数据切片前后补零
    current_data = np.array(zeros_oneside + list(current_data) + zeros_oneside)

    data_R.append(current_data)

data_all = data_W + data_L + data_D + data_R

# 预生成各分期的标签（与EEG采样点对齐）：0-清醒（W），1-浅睡（L），2-深睡（D），3-REM（R）
label_W = [0] * num_W
label_L = [1] * num_L
label_D = [2] * num_D
label_R = [3] * num_R
label_all = np.array(label_W + label_L + label_D + label_R)

# 验证集占总数据集的比例
percentage_validation = 0.2

# 训练集、验证集的切片数量
num_validation = int(np.floor(num_all * percentage_validation))
num_train = num_all - num_validation

# 打乱顺序，抽取验证集/训练集（数据+标签）
idx_all = np.arange(num_all)
np.random.shuffle(idx_all)
idx_validation = idx_all[:num_validation]
idx_train = idx_all[num_validation:]

data_train = []
label_train = []
for jj in idx_train:
    data_train.append(data_all[jj])
    label_train.append(label_all[jj])

data_validation = []
label_validation = []
for ii in idx_validation:
    data_validation.append(data_all[ii])
    label_validation.append(label_all[ii])

# 保存验证集/训练集数据（二进制）
np.savez('data_label_train', data_train=data_train, label_train=label_train)
np.savez('data_label_validation', data_validation=data_validation, label_validation=label_validation)

print('##### 数眠 DeepSleepNet 深度学习 #####')
print('【切片总数】')
print('清醒期：', num_W)
print('浅睡期：', num_L)
print('深睡期：', num_D)
print('REM期：', num_R)
print('【训练集 vs 验证集】')
print('验证集占比：', percentage_validation*100, '%')
print('训练集：', num_train)
print('验证集：', num_validation)

# 画5个示例切片
plt.figure
for idx in range(1, 5+1):
    plt.subplot(5, 1, idx)
    plt.plot(data_train[idx])
plt.show()



## 模型训练

In [ ]:
# 开始模型训练
print("===== Start Learning =====")
for epoch in range(epochs):
    trans_model.train()  # 切换为model训练模式
    sums = 0

    # 模型训练
    for ii in range(num_train):

        inputs = np.array(data_train[ii])
        labels = [label_train[ii]]

        # inputs = inputs.reshape(-1, 1, len_slice)
        inputs = inputs.reshape(-1, 1, len_slice)

        inputs = torch.tensor(inputs, dtype=torch.float)
        labels = torch.tensor(labels, dtype=torch.long)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # 初始化 optimizer - >初始化所有梯度
        logits = trans_model(inputs)  # logits是模型预测的值
        loss = criterion(logits, labels)  # 通过先前定义的cost函数计算loss
        loss.backward()  # 反向传播
        optimizer.step()

        train_loss[epoch] += loss.item()  # 训练过程的loss累加

        ps = F.softmax(logits, dim=1)  # 将logits归一化，赋给ps
        top_p, top_class = ps.topk(1, dim=1)  # 求出ps的最大值top_p及其索引top_class
        equals = top_class == labels.reshape(top_class.shape)   # 寻找正确分类项

        train_acc[epoch] += torch.mean(equals.type(torch.FloatTensor)).item()  # 计算训练过程的准确度

        sums += 1

    # 求整个训练过程中loss和acc的平均值
    train_loss[epoch] /= sums
    train_acc[epoch] /= sums

    # 模型验证
    val_sums = 0
    trans_model.eval()  # 切换为model验证模式

    with torch.no_grad():  # 张量的计算过程中无需计算梯度

        for jj in range(num_validation):

            inputs = data_validation[jj]
            labels = [label_validation[jj]]

            inputs = inputs.reshape(-1, 1, len_slice)

            inputs = torch.tensor(inputs, dtype=torch.float)
            labels = torch.tensor(labels, dtype=torch.long)
            inputs, labels = inputs.to(device), labels.to(device)

            # 计算预测值
            logits = trans_model(inputs)  # logits是模型预测的值
            val_single_loss = criterion(logits, labels)  # 通过先前定义的cost函数计算loss

            val_loss[epoch] += val_single_loss.item()  # 验证过程的loss累加

            # 计算准确度
            ps = F.softmax(logits, dim=1)  # 将logits归一化，赋给ps
            top_p, top_class = ps.topk(1, dim=1)  # 求出ps的最大值top_p及其索引top_class

            equals = top_class == labels.view(*top_class.shape)  # 寻找正确分类项
            val_acc[epoch] += torch.mean(equals.type(torch.FloatTensor)).item()  # 计算验证过程的准确度

            val_sums += 1

        # 求整个验证过程中loss和acc的平均值
        val_loss[epoch] /= val_sums
        val_acc[epoch] /= val_sums

    # PRINT LOSS & ACC
    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {train_loss[epoch]:.3f}.. "
          f"Train acc: {train_acc[epoch]:.3f}.. "
          f"val loss: {val_loss[epoch]:.3f}.. "
          f"val accuracy: {val_acc[epoch]:.3f}")

    #  保存最佳模型
    if val_loss[epoch] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, val_loss[epoch]))
        torch.save(trans_model.state_dict(), 'Trans_model_best.pt')
        valid_loss_min = val_loss[epoch]

        # 初始化“远端停止”计数（如果存在最低 Loss 值）
        cnt = 0

    # 如果 Loss 改进超过 30 次，则退出
    #  Early Stopping
    if cnt >= 30:
        print("Early Stopping")
        break

    cnt += 1  # Loss 改进失败